# 0. Prelude

## 0.1. Install and Import Packages

In [ ]:
# Install Required Packages
%pip install -r requirements.txt

In [48]:
# Zip File Download and Extraction
from io import BytesIO
from urllib.request import urlopen
from zipfile import ZipFile

# Census Data Extraction
from census import Census
import pandas as pd

# GeoTif file extraction
import geopandas as gpd
from shapely import Point
import rioxarray

#Utilize .env file for api key(s)
from dotenv import dotenv_values
import os


## 0.2. Files and URL Register

In [50]:
# File locations for windows file system.

transit_routes_url = 'https://seshat.datasd.org/gis_transit_routes/transit_routes_datasd.zip'
transit_stops_url = 'https://seshat.datasd.org/gis_transit_stops/transit_stops_datasd.zip'
california_tracts_url = 'https://www2.census.gov/geo/tiger/TIGER2021/TRACT/tl_2021_06_tract.zip'
us_counties_url = 'https://www2.census.gov/geo/tiger/TIGER2021/COUNTY/tl_2021_us_county.zip'
HRPD_url = 'https://data.humdata.org/dataset/eec3a01f-5237-4896-8059-a6be193ca964/resource/58b480a3-afbd-4d1e-bd9f-5ef480bffb61/download/population_usa28_-120_2019-07-01.tif.zip'

transit_routes_file = ".\\Data\\transit_routes\\"
transit_stops_file = ".\\Data\\transit_stops\\"
california_tracts_file = ".\\Data\\california_tracts\\"
us_counties_file = ".\\Data\\us_counties\\"
HRPD_file = '.\\Data\\HRPD\\'

transit_stops_file_name = 'transit_stops_datasd.shp'
california_tracts_file_name = 'tl_2021_06_tract.shp'
us_counties_file_name = 'tl_2021_us_county.shp'
HRPD_file_name = 'population_usa28_-120_2019-07-01.tif'
HRPD_sd_shp_file_name = 'HRPD_sd.shp'

census_file_name = '.\\Data\\census_data.csv'

## Loading Census API Key from .env File

In [100]:
env_path = f"{os.getcwd()}\\.env"
config = dotenv_values(env_path)

census_api_key = config["census_api_key"]

#Census object to be used later
c = Census(
            census_api_key,
            year=2021)

## 0.3. Load Local Datasets

In [52]:
# Only need to run Data Collection and Transformation once.
# after that run this chunk to load the data, so you dont 
# have to redownload and reprocess the data.

# This code chunck will be Placed lower in the final verison.

county_df = gpd\
    .read_file(us_counties_file + us_counties_file_name)\
    .to_crs("EPSG:4326")
county_df = county_df.loc[county_df['NAME']== 'San Diego',]

tract_df = gpd\
    .read_file(california_tracts_file + california_tracts_file_name)\
    .to_crs("EPSG:4326")
tract_df = tract_df.loc[tract_df['COUNTYFP']== '073',]

# 1. Data Collection and Transformation

## 1.1. Download & Unzip shape and raster files.

In [53]:
def download_unzip(file_url,file_location):
    with urlopen(file_url) as zipresp:
        with ZipFile(BytesIO(zipresp.read())) as zfile:
            zfile.extractall(file_location)

download_unzip(transit_routes_url,transit_routes_file)
download_unzip(transit_stops_url,transit_stops_file)
download_unzip(california_tracts_url,california_tracts_file)
download_unzip(us_counties_url,us_counties_file)
download_unzip(HRPD_url,HRPD_file)

## 1.2. Create Census Data Table

In [ ]:
name_list = ['Total'
             , 'Less than 5 mins'
             , '5 to 10 Mins'
             , '10 to 15 Mins'
             , '15 to 20 Mins'
             , '20 to 25 Mins'
             , '25 to 30 Mins'
             , '30 to 35 Mins'
             , '35 to 40 Mins'
             , '40 to 45 Mins'
             , '45 to 60 Mins'
             , '60 to 90 Mins'
             , 'More than 90 Mins'
             , 'Total Population'
             ]

key_list = ['B08303_0'+str(i).zfill(2)+'E' for i in range(1, len(name_list))]
key_list += [
    'B01003_001E',
    

]
key_list

In [74]:
#See https://api.census.gov/data/2021/acs/acs5/groups/B08303.html for available fields
socio_eco_census = c.acs5.state_county_tract(
    
    state_fips = "06",      # California
    county_fips = "073",    # San Diego
    tract = "*",            # All tracts
    fields = key_list       # All Estimate Fields in Coummute survery table., Population
)

In [75]:
census_df = pd.DataFrame(socio_eco_census)

### 1.2.1. Rename Columns to appropriate title

In [ ]:
column_rename = {key_list[i]: name_list[i] for i in range(0,len(key_list))}

census_df = census_df.rename(columns=column_rename)

census_df

### 1.2.2. Remove trivial columns and rows.

In [77]:
census_df = census_df.drop(columns=['state','county'], index=[736])

### 1.2.3. Add Mean Time Column

In [78]:
commute_mean_time = [0 for x in range(736)]
for i in range(1,10):
    commute_mean_time = commute_mean_time + \
        census_df[name_list[i]]*(2.5+(i-1)*5)

commute_mean_time = commute_mean_time +  census_df[name_list[10]]*52.75
commute_mean_time = commute_mean_time +  census_df[name_list[11]]*75
commute_mean_time = commute_mean_time/census_df['Total']

# Ignoring 90 mins for now.
# commute_mean_time.mean()

census_df['Mean Time'] = commute_mean_time

### 1.2.4. Save Dataframe to CSV file

In [ ]:
census_df.to_csv(census_file_name,index=False)
census_df

## 1.3. Convert HRPD Raster Data to Shapefile

In [93]:
san_deigo_polygon = county_df.loc[539,'geometry']

# Takes some Time (35sec on desktop)
HRPD_tif = rioxarray.open_rasterio(HRPD_file + HRPD_file_name, cache = False)
HRPD_sd_tif = HRPD_tif.rio.clip([san_deigo_polygon])
HRPD_tif = HRPD_sd_tif.rio.reproject("EPSG:4326", cache = False)


In [94]:
HRPD_sd_df = HRPD_sd_tif[0].to_dataframe('Population').dropna()

# Takes some Time (10sec on desktop)
HRPD_sd_df['geometry'] = [Point(index[1],index[0]) for index,row in HRPD_sd_df.iterrows()]

HRPD_sd_df = gpd.GeoDataFrame(HRPD_sd_df['Population'],
                              geometry = HRPD_sd_df['geometry'])\
                                .reset_index(drop=True)

In [ ]:
# Takes some Time (25sec on desktop)
HRPD_sd_df.to_file(HRPD_file + HRPD_sd_shp_file_name, 
                   index=False,
                   crs='EPSG:4326')
HRPD_sd_df

## 1.4. Create other paramerters by processing data

### 1.4.1. Create Pop Centers, tract Pop and Distance from Hub

In [96]:
def pop_center(pop_table):
    x = 0
    y = 0
    pop_total = 0
    for index,row in pop_table.iterrows():
        x += row['geometry'].x*row['Population']
        y += row['geometry'].y*row['Population']
        pop_total += row['Population']

    return Point(x/pop_total ,y/pop_total)

tract_df['tract'] = pd.to_numeric(tract_df['TRACTCE'])
tract_df = tract_df[tract_df['tract'] != 990100]

# Takes some Time (50sec on desktop)
pop_center_list = [pop_center(HRPD_sd_df.clip(row['geometry'])) for index, row in tract_df.iterrows()]


### 1.4.2. Create Stops and Route Length in Tracts

In [ ]:
transit_stops_df = gpd.read_file(transit_stops_file + transit_stops_file_name).to_crs("EPSG:4326")
# Remember to Buffer the tract polygons, so the correctly include the stops around them
transit_stops_df

In [ ]:
transit_routes_df = gpd.read_file(transit_routes_file + 'transit_routes_datasd.shp').to_crs("EPSG:4326")
# Remember to Buffer the tract polygons, so the correctly include the stops around them
transit_routes_df

# 2. Regression Models

# 3. Scratch Code for Testing

In [ ]:
#You can request an API key from https://api.census.gov/data/key_signup.html
API_KEY = '9e94761d2b2fb214be8b88bdaeccc47b8edded35'

c = Census(API_KEY,
           year=2021)

#See https://api.census.gov/data/2021/acs/acs5/groups/B08303.html for available fields
population_census = c.acs5.state_county_tract(
    
    state_fips = "06",      # California
    county_fips = "073",    # San Diego
    tract = "*",            # All tracts
    fields = 'B01003_001E', # Total Population
)
population_df = pd.DataFrame(population_census)
population_df = population_df.drop(columns=['state','county'], index=[736])
population_df = population_df.rename(columns={'B01003_001E': 'Total Population'})
population_df

In [25]:
population_df['Total Population'].sum()

3296317.0

In [26]:
HRPD_sd_df['Population'].sum()

2782294.505288508

In [13]:
def pop_center(pop_table):
    x = 0
    y = 0
    pop_total = 0
    for index,row in pop_table.iterrows():
        x += row['geometry'].x*row['Population']
        y += row['geometry'].y*row['Population']
        pop_total += row['Population']

    return Point(x/pop_total ,y/pop_total)

tract_df['tract'] = pd.to_numeric(tract_df['TRACTCE'])
tract_df = tract_df[tract_df['tract'] != 990100]

# Takes some Time (50sec on desktop)
pop_center_list = gpd.GeoSeries([pop_center(HRPD_sd_df.clip(row['geometry'])) for index, row in tract_df.iterrows()])

In [ ]:
import matplotlib.colors as colors
import matplotlib.pyplot as plt

fig,ax = plt.subplots(1,1,sharex=True,sharey=True,figsize=(16,12))
tract_df.boundary.plot(ax=ax,linewidth=0.2)
HRPD_sd_df.plot(column='Population',
                           ax=ax,
                           markersize=0.01,
                           cmap='YlGnBu',
                           legend=True,
                           norm=colors.LogNorm(0.0029,520)
                           )
transit_stops_df.plot(ax=ax,color='black',markersize=0.1)
gpd.GeoSeries(pop_center_list).plot(ax=ax, markersize=1, color='red')
plt.title("HRPD - San Diego, with pop centers",
          fontsize=25)